In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import string
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction import text


In [11]:
import os
os.listdir('../data/')

['__init__.py', 'datafile', 'dataloader.py', '__pycache__']

In [3]:
train = pd.read_csv('')

FileNotFoundError: [Errno 2] File b'/data/datafile/Train.csv' does not exist: b'/data/datafile/Train.csv'

In [4]:
train.head()

,text,sentiment
0,For a movie that gets no respect there sure ar...,0
1,Bizarre horror movie filled with famous faces ...,0
2,"A solid, if unremarkable film. Matthau, as Ein...",0
3,It's a strange feeling to sit alone in a theat...,0
4,"You probably all already know this by now, but...",0


In [5]:
train.dropna(inplace=True)

In [13]:
lemmatizer = WordNetLemmatizer() 
def lemmatize(phrase):
    filtered = []
    word_list = phrase.split()
    filtered = [
        lemmatizer.lemmatize(word).lower()
        for word in word_list 
        if word not in string.punctuation
    ]
    return " ".join(filtered)

In [14]:
train['text'] = train['text'].apply(lambda x:lemmatize(x))

In [16]:
train["word_count"] = train['text'].apply(lambda x: len(x))
train.drop(train[train["word_count"] == 0].index, inplace=True)


In [17]:
train.reset_index(drop=True, inplace=True)


In [38]:
from collections import defaultdict
def filter_word(phrases):
    phrase_sentiment = defaultdict(int)
    for phrase in phrases:
        for word in phrase.split():
            phrase_sentiment[word] += 1
    return phrase_sentiment



def intersection(series_list):
    intersect = set(series_list[0])
    for series in series_list[1:]:
        intersect.intersection_update(series)
        
    comm_token = filter_word(intersect)
    _tokens = pd.Series([x for x in comm_token.keys()])
    print(_tokens.shape)
    return _tokens

def stopword():
    sentiments = train['sentiment'].unique()
    print(sentiments)
    series_list = []
    for sentiment in sentiments:
        phrase_sentiment = filter_word(
            train.loc[(train['sentiment'] == sentiment), "text"].values
            )
        series = pd.Series(
            [key for key in phrase_sentiment.keys()]
            )
        series_list.append(series)

    common = intersection(series_list)
    my_stop_word = text.ENGLISH_STOP_WORDS.union(common)
    return my_stop_word

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split 

In [40]:
my_stop_word = stopword()
def my_preprocessor(doc):
    phrase = doc.split()
    for i in range(0,len(phrase)):
        k = phrase.pop(0)
        if k not in my_stop_word:
            phrase.append(k)
            
#     print(phrase)
    return ' '.join(phrase)

[0 1]
(63554,)


In [54]:
tf=TfidfVectorizer(analyzer='word',ngram_range=(1,2),stop_words='english')


In [55]:
tf_train = tf.fit_transform(train['text'])
X_train , X_test , y_train , y_test = train_test_split(tf_train,train['sentiment'] ,test_size =0.3,
                                                       random_state=123)

In [56]:
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X_train , y_train)
predict = rf.predict(X_test)
print('Accuracy',f1_score(y_test,predict,average='weighted'))

/home/abhaik/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy 0.7579668635438308


frozenset({'obscure,',
           'angsty',
           'filming.',
           'differences,',
           'mustache,',
           'connery.',
           'outstanding.',
           "timon's",
           'justifies',
           'location.',
           'screen)',
           'happy-go-lucky',
           'motorist',
           'mutiny"',
           'strangelove".',
           'troopers',
           "mcbain's",
           "catherine's",
           'daredevil,',
           'party",',
           '(probably)',
           'hills',
           'surprises!',
           'tasty',
           'capture.',
           '{grade:',
           'closeup',
           'jerry.',
           'all.)',
           'non-japanese',
           '"screw',
           'mayberry-like',
           'scalpel',
           "wallace's",
           'penny,',
           'fuqua',
           'panel',
           'roadie',
           'diversity,',
           'played,',
           '"letting',
           'breath-takingly',
           'dudik